### Library imports

In [20]:
from azure.identity import DefaultAzureCredential
from azure.ai.ml import MLClient, command
from azure.ai.ml.dsl import pipeline 
from azure.ai.ml.entities import JobSchedule, RecurrencePattern, RecurrenceTrigger
from azureml.pipeline.core import TimeZone

import datetime

ModuleNotFoundError: No module named 'azureml.pipeline'

### 1. Get handle to workspace

In [ ]:
ml_client = MLClient(
    DefaultAzureCredential(),
    subscription_id="27a6aae6-ce60-4ae4-a06e-cfe9c1e824d4",
    resource_group_name="RG-ADA-MLOPS-POC",
    workspace_name="azu-ml-ada-mlops-poc",
)

### 2. Configure the data drift detection script as a pipeline job

In [19]:
from data_drift.data_drift_check import data_drift_component

@pipeline(default_compute = 'shared-compute-poc', allow_reuse = False)

def data_drift_pipeline_definition():
    dd_node = data_drift_component()

pipeline_job = data_drift_pipeline_definition()

ModuleNotFoundError: No module named 'azureml.datadrift'

### 3. Create the schedule

In [ ]:
schedule_name = "data-dift-schedule"

recurrence_trigger = RecurrenceTrigger(
    frequency="hour",
    interval=4,
    time_zone=TimeZone.UTC,
)

job_schedule = JobSchedule(
    name=schedule_name, trigger=recurrence_trigger, create_job=pipeline_job
)

### 4. Start the schedule

In [ ]:
job_schedule = ml_client.schedules.begin_create_or_update(
    schedule=job_schedule
).result()
print(job_schedule)